# Verifying a Neural Network

This notebook demonstrates how to represent a shallow Neural Network (NN) and how to use Imandra's verification capabilities to prove or disprove properties of this Network.

The NN used has been trained on the Iris dataset and is composed of 2 layers :
1. a single-neuron layer with ReLu activation function
2. a 3-neuron layer with softmax activation function for the output

![](neural_net_ex.png)

In [1]:
(* set pretty printer *)
let pp_approx fmt r = CCFormat.fprintf fmt "%s" (Real.to_string_approx r) [@@program]
#install_printer pp_approx

val pp_approx : Format.formatter -> real -> unit = <fun>


 ## 1. Pre-processing data
 We first define a custom type adapted to our specific dataset to manipulate the data more conveniently.
 We then define a function that converts this custom type into a tuple of Real numbers. If we needed to make any data normalisation, we would do it in the `process_iris_input` function.

In [ ]:
(* create custom input type *)

open Real

type iris_input = {
    sepal_len: real;
    sepal_width: real;
    petal_len: real;
    petal_width: real;
}

let process_iris_input (x: iris_input) =
    let f0 = x.sepal_len in
    let f1 = x.sepal_width in
    let f2 = x.petal_len in
    let f3 = x.petal_width in
    (f0, f1, f2, f3);;

## Representing the model
We then define functions that represent the NN model.

### Layers

The first layer `layer_0` takes as parameters an activation function, a set of weights and a set of inputs in order to demonstrate Imandra's reasoning capabilities.

The second layer `layer_1` has its weights hard-coded. It does not have an activation function because the original `softmax` function uses exponentiation, which is not supported in Imandra; instead, an equivalent of `argmax` is used in the next step to obtain the classification result.

In [35]:
(* define layer 0 function *)

let layer_0 activation (w0, w1, w2, w3, w4) (f1, f2, f3, f4) =
    activation (w0 +. w1 *. f1 +. w2 *.f2 +. w3 *. f3 +. w4 *. f4)

(* define layer 1 function *)
let layer_1 (f1) =
    let o1 = -2.651993 *. f1 + 0.81521773 in  
    let o2 = -0.83343804 *. f1 +. 0.27192873 in
    let o3 = -0.27463955 +. -1.21521 in
    (o1, o2, o3);;

val layer_0 : (t -> 'a) -> t * t * t * t * t -> t * t * t * t -> 'a = <fun>
val layer_1 : t -> t * t * t = <fun>


### Output

We then define a function that prints out the classification result. 

In [36]:
(* print result *)
let process_iris_output (c0, c1, c2) =
    if (c0 >=. c1) && (c0 >=. c2) then "setosa"
    else if (c1 >=. c0) && (c1 >=. c2) then "versicolor"
    else "virginica"

val process_iris_output : t * t * t -> string = <fun>


### Putting things together

We chain the previously defined functions together to obtain an executable model. It takes an activation function, a set of weights for the first layer and input data as inputs, and outputs a string representing the predicted class.

In most of the following examples, the weights and activation function will always be the same so for the sake of readability, we also define `model_relu`, which only takes a data sample as input and outputs its predicted class.

If we're always going to use the same weights and activation fn, why did we even bother parametrising them? This will be shown in a couple of cells.

In [38]:
(* put model together *)

let model activation weights input = process_iris_input input |> layer_0 activation weights |> layer_1 |> process_iris_output;;

let relu x = 
    if x <. 0. 
    then 0. 
    else x;;
    
let weights_0 = (1.0023211, 1.1538234, -0.30127743, 0.9319558, 2.179688)

(* Example of partial function application *)
let model_relu = model relu weights_0;;

val model : (t -> t) -> t * t * t * t * t -> iris_input -> string = <fun>
val relu : t -> t = <fun>
val weights_0 : t * t * t * t * t =
  (1.0023211, 1.1538234, -0.30127743, 0.9319558, 2.179688)
val model_relu : iris_input -> string = <fun>


Let us test our model with an instance of "versicolor" taken from our normalised testing set.

In [39]:
(* Example with a versicolor instance, normalized input *)
let input = {
    sepal_len = 0.31099753;
    sepal_width = -0.59237301;
    petal_len = 0.53540856;
    petal_width = 0.00087755;
};;

model_relu input

val input : iris_input =
  {sepal_len = 0.31099753; sepal_width = -0.59237301; petal_len = 0.53540856;
   petal_width = 0.00087755}
- : string = "versicolor"


## Reasoning with Imandra

### Instance
`instance` takes a predicate as input, and says if variables exist that satisfy this predicate. 

The predicate is a function that takes variables as input, and returns a boolean value.

Imandra says not only if an instance exists or not, but also gives an example value.

First, we prove that no instance exists that is classfied as "foo". Then, we prove that (at least) one input exists that is classified as "virginica".  

In [40]:
instance (fun x -> model_relu x = "foo")

- : iris_input -> bool = <fun>


Unsatisfiable proof call graph proof summary full graph ground_instances 0 definitions 0 inductions 0 search_time 0.016s details Expand smt_stats rlimit count 1052 mk bool var 1 num allocs 1272804953 memory 31.140000 max memory 31.140000 Expand start[0.016s]
 let (_x_0 : real)
 = 10023211/10000000 +. 5769117/5000000 *. …
 +. -30127743/100000000 *. … +. 4659779/5000000 *. …
 +. 272461/125000 *. …
 in
 let (_x_1 : real) = if _x_0 <. 0 then 0 else _x_0 in
 let (_x_2 : real) = -20835951/25000000 *. _x_1 +. 27192873/100000000 in
 let (_x_3 : real) = -2651993/1000000 *. _x_1 +. 81521773/100000000 in
 let (_x_4 : real) = -5492791/20000000 +. -121521/100000 in
 (if _x_3 >=. _x_2 && _x_3 >=. _x_4 then "setosa"
 else if _x_2 >=. _x_3 && _x_2 >=. _x_4 then "versicolor" else "virginica")
 = "foo" simplify into let (_x_0 : real) = 5769117/5000000 *. :var_0:.sepal_len in
let (_x_1 : real) = -30127743/100000000 *. :var_0:.sepal_width in
let (_x_2 : real) = 4659779/5000000 *. :var_0:.petal_len in
let (_x_3 : real) = 272461/125000 *. :var_0:.petal_width in
let (_x_4 : real)
 = if -10023211/10000000 <=. (_x_0 +. _x_1 +. _x_2 +. _x_3)
 then 10023211/10000000 +. _x_0 +. _x_1 +. _x_2 +. _x_3 else 0
in
let (_x_5 : real) = 5769117/5000000 *. … in
let (_x_6 : real) = -30127743/100000000 *. … in
let (_x_7 : real) = 4659779/5000000 *. … in
let (_x_8 : real) = 272461/125000 *. … in
(if (if -10023211/10000000 <=. (_x_5 +. _x_6 +. _x_7 +. _x_8)
 then 10023211/10000000 +. _x_5 +. _x_6 +. _x_7 +. _x_8 else 0)
 <=. 13582225/45463874 && _x_4 <=. 57626682/66299825
 then "setosa"
 else
 if _x_4 >=. 13582225/45463874 && _x_4 <=. 44044457/20835951
 then "versicolor" else "virginica")
= "foo" expansions [] rewrite_steps forward_chaining Unsat digraph "proof" {
p_267 [label="Start (let (_x_0 : real)\l = 10023211/10000000 +. 5769117/5000000 *. …\l +. -30127743/100000000 *. … +. 4659779/5000000 *. …\l +. 272461/125000 *. …\l in\l let (_x_1 : real) = if _x_0 \<. 0 then 0 else _x_0 in\l let (_x_2 : real) = -20835951/25000000 *. _x_1 +. 27192873/100000000\l in\l let (_x_3 : real) = -2651993/1000000 *. _x_1 +. 81521773/100000000 in\l let (_x_4 : real) = -5492791/20000000 +. -121521/100000 in\l (if _x_3 \>=. _x_2 && _x_3 \>=. _x_4 then \"setosa\"\l else\l if _x_2 \>=. _x_3 && _x_2 \>=. _x_4 then \"versicolor\" else \"virginica\")\l = \"foo\" :time 0.016s)",shape=box,style=filled,fontname="courier",fontsize=14];
p_267 -> p_266 [label=""];
p_266 [label="Simplify (let (_x_0 : real) = 5769117/5000000 *. :var_0:.sepal_len in\l let (_x_1 : real) = -30127743/100000000 *. :var_0:.sepal_width in\l let (_x_2 : real) = 4659779/5000000 *. :var_0:.petal_len in\l let (_x_3 : real) = 272461/125000 *. :var_0:.petal_width in\l let (_x_4 : real)\l = if -10023211/10000000 \<=. (_x_0 +. _x_1 +. _x_2 +. _x_3)\l then 10023211/10000000 +. _x_0 +. _x_1 +. _x_2 +. _x_3 \l else 0\l in\l let (_x_5 : real) = 5769117/5000000 *. … in\l let (_x_6 : real) = -30127743/100000000 *. … in\l let (_x_7 : real) = 4659779/5000000 *. … in\l let (_x_8 : real) = 272461/125000 *. … in\l (if (if -10023211/10000000 \<=. (_x_5 +. _x_6 +. _x_7 +. _x_8)\l then 10023211/10000000 +. _x_5 +. _x_6 +. _x_7 +. _x_8 \l else 0)\l \<=. 13582225/45463874 && _x_4 \<=. 57626682/66299825\l then \"setosa\"\l else\l if _x_4 \>=. 13582225/45463874 && _x_4 \<=. 44044457/20835951\l then \"versicolor\" else \"virginica\")\l = \"foo\" :expansions [] :rw [] :fc [])",shape=box,style=filled,fontname="courier",fontsize=14];
p_266 -> p_265 [label=""];
p_265 [label="Unsat",shape=box,style=filled,fontname="courier",fontsize=14];
}
 Load graph Loading.. digraph "call graph" {
goal [label="let (_x_0 : real) = 5769117/5000000 *. x.sepal_len in\llet (_x_1 : real) = -30127743/100000000 *. x.sepal_width in\llet (_x_2 : real) = 4659779/5000000 *. x.petal_len in\llet (_x_3 : real) = 272461/125000 *. x.petal_width in\llet (_x_4 : real)\l = if -10023211/10000000 \<=. (_x_0 +. _x_1 +. _x_2 +. _x_3)\l then 10023211/10000000 +. _x_0 +. _x_1 +. _x_2 +. _x_3 el

In [41]:
(* Prove that an instance exists *)
instance (fun x -> model_relu x = "virginica")

- : iris_input -> bool = <fun>
module CX : sig val x : iris_input end


Instance (after 0 steps, 0.018s):
 let (x : iris_input) =
 {sepal_len = 8855.0; sepal_width = 0.0; petal_len = 2437.0;
 petal_width = (Real.mk_of_string "-371658922820659111/64879817661840")}

Instance call graph proof digraph "call graph" {
goal [label="let (_x_0 : real) = 5769117/5000000 *. x.sepal_len in\llet (_x_1 : real) = -30127743/100000000 *. x.sepal_width in\llet (_x_2 : real) = 4659779/5000000 *. x.petal_len in\llet (_x_3 : real) = 272461/125000 *. x.petal_width in\llet (_x_4 : real)\l = if -10023211/10000000 \<=. (_x_0 +. _x_1 +. _x_2 +. _x_3)\l then 10023211/10000000 +. _x_0 +. _x_1 +. _x_2 +. _x_3 else 0\lin\l(if _x_4 \<=. 13582225/45463874 && _x_4 \<=. 57626682/66299825 then \"setosa\"\l else\l if _x_4 \>=. 13582225/45463874 && _x_4 \<=. 44044457/20835951\l then \"versicolor\" else \"virginica\")\l= \"virginica\"",shape=box,style=filled,color="cyan",fontname="courier",fontsize=14];
}
 Load graph Loading.. proof attempt summary full graph ground_instances 0 definitions 0 inductions 0 search_time 0.018s details Expand smt_stats num checks 2 arith-make-feasible 1 arith-max-columns 9 rlimit count 1484 mk clause 16 datatype occurs check 1 mk bool var 15 decisions 1 arith-propagations 2 propagations 6 arith-bound-propagations-cheap 2 arith-max-rows 1 datatype accessor ax 1 datatype constructor ax 1 num allocs 1347452151 final checks 1 added eqs 6 arith-upper 3 memory 36.300000 max memory 36.690000 Expand start[0.018s]
 let (_x_0 : real)
 = 10023211/10000000 +. 5769117/5000000 *. …
 +. -30127743/100000000 *. … +. 4659779/5000000 *. …
 +. 272461/125000 *. …
 in
 let (_x_1 : real) = if _x_0 <. 0 then 0 else _x_0 in
 let (_x_2 : real) = -20835951/25000000 *. _x_1 +. 27192873/100000000 in
 let (_x_3 : real) = -2651993/1000000 *. _x_1 +. 81521773/100000000 in
 let (_x_4 : real) = -5492791/20000000 +. -121521/100000 in
 (if _x_3 >=. _x_2 && _x_3 >=. _x_4 then "setosa"
 else if _x_2 >=. _x_3 && _x_2 >=. _x_4 then "versicolor" else "virginica")
 = "virginica" simplify into let (_x_0 : real) = 5769117/5000000 *. … in
let (_x_1 : real) = -30127743/100000000 *. … in
let (_x_2 : real) = 4659779/5000000 *. … in
let (_x_3 : real) = 272461/125000 *. … in
let (_x_4 : real)
 = if -10023211/10000000 <=. (_x_0 +. _x_1 +. _x_2 +. _x_3)
 then 10023211/10000000 +. _x_0 +. _x_1 +. _x_2 +. _x_3 else 0
in
(if _x_4 <=. 13582225/45463874 && _x_4 <=. 57626682/66299825 then "setosa"
 else
 if _x_4 >=. 13582225/45463874 && _x_4 <=. 44044457/20835951
 then "versicolor" else "virginica")
= "virginica" expansions [] rewrite_steps forward_chaining Sat (Some let (x : iris_input) =
 {sepal_len = 8855.0; sepal_width = 0.0; petal_len = 2437.0;
 petal_width = (Real.mk_of_string "-371658922820659111/64879817661840")}
) digraph "proof" {
p_270 [label="Start (let (_x_0 : real)\l = 10023211/10000000 +. 5769117/5000000 *. …\l +. -30127743/100000000 *. … +. 4659779/5000000 *. …\l +. 272461/125000 *. …\l in\l let (_x_1 : real) = if _x_0 \<. 0 then 0 else _x_0 in\l let (_x_2 : real) = -20835951/25000000 *. _x_1 +. 27192873/100000000\l in\l let (_x_3 : real) = -2651993/1000000 *. _x_1 +. 81521773/100000000 in\l let (_x_4 : real) = -5492791/20000000 +. -121521/100000 in\l (if _x_3 \>=. _x_2 && _x_3 \>=. _x_4 then \"setosa\"\l else\l if _x_2 \>=. _x_3 && _x_2 \>=. _x_4 then \"versicolor\" else \"virginica\")\l = \"virginica\" :time 0.018s)",shape=box,style=filled,fontname="courier",fontsize=14];
p_270 -> p_269 [label=""];
p_269 [label="Simplify (let (_x_0 : real) = 5769117/5000000 *. … in\l let (_x_1 : real) = -30127743/100000000 *. … in\l let (_x_2 : real) = 4659779/5000000 *. … in\l let (_x_3 : real) = 272461/125000 *. … in\l let (_x_4 : real)\l = if -10023211/10000000 \<=. (_x_0 +. _x_1 +. _x_2 +. _x_3)\l then 10023211/10000000 +. _x_0 +. _x_1 +. _x_2 +. _x_3 \l else 0\l in\l (if _x_4 \<=. 13582225/45463874 && _x_4 \<=. 57626682/66299825\l then \"setosa\"\l else\l if _x_4 \>=. 13582225/45463874 && _x_4 \<=. 44044457/20835951\l then \"versicolor\" else \"virginica\")\l = \"virginica\" :expansions [] :rw [] :fc [])",shape=box,style=filled,fontname="courier",fontsize=14];
p_269 -> p_268 [label=""];
p_268 [label="Sat (Some let (x : iris_input) =\l \{sepal_len

Imandra gives an example, but as we normalised our data, the extreme values given are not helpful. We can compose additional constraints, for instance specifying that the data is in the model's domain. 

In [47]:
(* add input constraints *)
let is_valid_input input =
  -3. <=. input.sepal_len && input.sepal_len <=. 3. &&
  -3. <=. input.sepal_width && input.sepal_width <=. 3. &&
  -3. <=. input.petal_len && input.petal_len <=. 3. &&
  -3. <=. input.petal_width && input.petal_width <=. 3.

instance (fun x -> model_relu x = "versicolor" && is_valid_input x)

val is_valid_input : iris_input -> bool = <fun>
- : iris_input -> bool = <fun>
module CX : sig val x : iris_input end


Instance (after 0 steps, 0.017s):
 let (x : iris_input) =
 {sepal_len = 0.0; sepal_width = -1.0; petal_len = 0.0; petal_width = 0.0}

Instance call graph proof digraph "call graph" {
goal [label="let (_x_0 : real) = x.sepal_len in\llet (_x_1 : real) = 5769117/5000000 *. _x_0 in\llet (_x_2 : real) = x.sepal_width in\llet (_x_3 : real) = -30127743/100000000 *. _x_2 in\llet (_x_4 : real) = x.petal_len in\llet (_x_5 : real) = 4659779/5000000 *. _x_4 in\llet (_x_6 : real) = x.petal_width in\llet (_x_7 : real) = 272461/125000 *. _x_6 in\llet (_x_8 : real)\l = if -10023211/10000000 \<=. (_x_1 +. _x_3 +. _x_5 +. _x_7)\l then 10023211/10000000 +. _x_1 +. _x_3 +. _x_5 +. _x_7 else 0\lin\l((((((((if _x_8 \<=. 13582225/45463874 && _x_8 \<=. 57626682/66299825\l then \"setosa\"\l else\l if _x_8 \>=. 13582225/45463874 && _x_8 \<=. 44044457/20835951\l then \"versicolor\" else \"virginica\")\l = \"versicolor\" && -3 \<=. _x_0)\l && _x_0 \<=. 3)\l && -3 \<=. _x_2)\l && _x_2 \<=. 3)\l && -3 \<=. _x_4)\l && _x_4 \<=. 3)\l && -3 \<=. _x_6)\l&& _x_6 \<=. 3",shape=box,style=filled,color="cyan",fontname="courier",fontsize=14];
}
 Load graph Loading.. proof attempt summary full graph ground_instances 0 definitions 0 inductions 0 search_time 0.017s details Expand smt_stats num checks 2 arith-assume-eqs 2 arith-make-feasible 5 arith-max-columns 11 rlimit count 2094 mk clause 10 datatype occurs check 2 mk bool var 27 arith-lower 7 decisions 4 arith-propagations 2 propagations 4 interface eqs 2 arith-bound-propagations-cheap 2 arith-max-rows 3 datatype accessor ax 1 datatype constructor ax 1 num allocs 1519933079 final checks 3 added eqs 8 arith eq adapter 2 arith-upper 9 memory 42.430000 max memory 42.800000 Expand start[0.017s]
 let (_x_0 : real)
 = 10023211/10000000 +. 5769117/5000000 *. …
 +. -30127743/100000000 *. … +. 4659779/5000000 *. …
 +. 272461/125000 *. …
 in
 let (_x_1 : real) = if _x_0 <. 0 then 0 else _x_0 in
 let (_x_2 : real) = -20835951/25000000 *. _x_1 +. 27192873/100000000 in
 let (_x_3 : real) = -2651993/1000000 *. _x_1 +. 81521773/100000000 in
 let (_x_4 : real) = -5492791/20000000 +. -121521/100000 in
 let (_x_5 : real) = :var_0:.petal_len in
 let (_x_6 : real) = :var_0:.petal_width in
 (if _x_3 >=. _x_2 && _x_3 >=. _x_4 then "setosa"
 else if _x_2 >=. _x_3 && _x_2 >=. _x_4 then "versicolor" else "virginica")
 = "versicolor"
 && -3 <=. …
 && … <=. 3
 && -3 <=. …
 && … <=. 3
 && -3 <=. _x_5 && _x_5 <=. 3 && -3 <=. _x_6 && _x_6 <=. 3 simplify into let (_x_0 : real) = :var_0:.sepal_len in
let (_x_1 : real) = 5769117/5000000 *. _x_0 in
let (_x_2 : real) = :var_0:.sepal_width in
let (_x_3 : real) = -30127743/100000000 *. _x_2 in
let (_x_4 : real) = :var_0:.petal_len in
let (_x_5 : real) = 4659779/5000000 *. _x_4 in
let (_x_6 : real) = :var_0:.petal_width in
let (_x_7 : real) = 272461/125000 *. _x_6 in
let (_x_8 : real)
 = if -10023211/10000000 <=. (_x_1 +. _x_3 +. _x_5 +. _x_7)
 then 10023211/10000000 +. _x_1 +. _x_3 +. _x_5 +. _x_7 else 0
in
((((((((if _x_8 <=. 13582225/45463874 && _x_8 <=. 57626682/66299825
 then "setosa"
 else
 if _x_8 >=. 13582225/45463874 && _x_8 <=. 44044457/20835951
 then "versicolor" else "virginica")
 = "versicolor" && -3 <=. _x_0)
 && _x_0 <=. 3)
 && -3 <=. _x_2)
 && _x_2 <=. 3)
 && -3 <=. _x_4)
 && _x_4 <=. 3)
 && -3 <=. _x_6)
&& _x_6 <=. 3 expansions [] rewrite_steps forward_chaining Sat (Some let (x : iris_input) =
 {sepal_len = 0.0; sepal_width = -1.0; petal_len = 0.0; petal_width = 0.0}
) digraph "proof" {
p_279 [label="Start (let (_x_0 : real)\l = 10023211/10000000 +. 5769117/5000000 *. …\l +. -30127743/100000000 *. … +. 4659779/5000000 *. …\l +. 272461/125000 *. …\l in\l let (_x_1 : real) = if _x_0 \<. 0 then 0 else _x_0 in\l let (_x_2 : real) = -20835951/25000000 *. _x_1 +. 27192873/100000000\l in\l let (_x_3 : real) = -2651993/1000000 *. _x_1 +. 81521773/100000000 in\l let (_x_4 : real) = -5492791/20000000 +. -121521/100000 in\l let (_x_5 : real) = :var_0:.petal_len in\l let (_x_6 : real) = :var_0:.petal_width in\l (if _x_3 \>=. _x_2 && _x_3 \>=. _x_4 then \"setosa\"\l else\l if _x_2 \>=. _x_3 && _x_2 \>=. _x_4 then \"versicolor\" el

Here we use the `model` defined earlier with parametrised weights and activation function in order for Imandra to reason about these parameters. 

In [31]:
(* reason about activation function *)
instance (fun f -> (model f weights_0 input = "versicolor"))

- : (t -> t) -> bool = <fun>
module CX : sig val f : 'a -> t end


Instance (after 0 steps, 0.016s):
 let f = function
 | _ -> (Real.mk_of_string "59046099/45463874")

Instance call graph proof digraph "call graph" {
goal [label="let (_x_0 : real) = sko_f_0 20405158435764143/10000000000000000 in\l(if _x_0 \<=. 13582225/45463874 && _x_0 \<=. 57626682/66299825 then \"setosa\"\l else\l if _x_0 \>=. 13582225/45463874 && _x_0 \<=. 44044457/20835951\l then \"versicolor\" else \"virginica\")\l= \"versicolor\"",shape=box,style=filled,color="cyan",fontname="courier",fontsize=14];
}
 Load graph Loading.. proof attempt summary full graph ground_instances 0 definitions 0 inductions 0 search_time 0.016s details Expand smt_stats num checks 2 arith-make-feasible 2 arith-max-columns 6 rlimit count 577 mk clause 2 mk bool var 7 arith-lower 1 decisions 1 arith-propagations 1 propagations 1 arith-bound-propagations-cheap 1 arith-max-rows 1 num allocs 907699739 final checks 1 added eqs 1 arith-upper 2 memory 21.670000 max memory 22.080000 Expand start[0.016s]
 let (_x_0 : real)
 = sko_f_0
 (….0 +. ….1 *. ….0 +. ….2 *. ….1 +. ….3 *. ….2
 +. ….4 *. ….3)
 in
 let (_x_1 : real) = -20835951/25000000 *. _x_0 +. 27192873/100000000 in
 let (_x_2 : real) = -2651993/1000000 *. _x_0 +. 81521773/100000000 in
 let (_x_3 : real) = -5492791/20000000 +. -121521/100000 in
 (if _x_2 >=. _x_1 && _x_2 >=. _x_3 then "setosa"
 else if _x_1 >=. _x_2 && _x_1 >=. _x_3 then "versicolor" else "virginica")
 = "versicolor" simplify into let (_x_0 : real) = sko_f_0 20405158435764143/10000000000000000 in
(if _x_0 <=. 13582225/45463874 && _x_0 <=. 57626682/66299825 then "setosa"
 else
 if _x_0 >=. 13582225/45463874 && _x_0 <=. 44044457/20835951
 then "versicolor" else "virginica")
= "versicolor" expansions [] rewrite_steps forward_chaining Sat (Some let f = function
 | _ -> (Real.mk_of_string "59046099/45463874")
 
) digraph "proof" {
p_249 [label="Start (let (_x_0 : real)\l = sko_f_0\l (….0 +. ….1 *. ….0 +. ….2 *. ….1 +. ….3 *. ….2\l +. ….4 *. ….3)\l in\l let (_x_1 : real) = -20835951/25000000 *. _x_0 +. 27192873/100000000\l in\l let (_x_2 : real) = -2651993/1000000 *. _x_0 +. 81521773/100000000 in\l let (_x_3 : real) = -5492791/20000000 +. -121521/100000 in\l (if _x_2 \>=. _x_1 && _x_2 \>=. _x_3 then \"setosa\"\l else\l if _x_1 \>=. _x_2 && _x_1 \>=. _x_3 then \"versicolor\" else \"virginica\")\l = \"versicolor\" :time 0.016s)",shape=box,style=filled,fontname="courier",fontsize=14];
p_249 -> p_248 [label=""];
p_248 [label="Simplify (let (_x_0 : real) = sko_f_0 20405158435764143/10000000000000000 in\l (if _x_0 \<=. 13582225/45463874 && _x_0 \<=. 57626682/66299825\l then \"setosa\"\l else\l if _x_0 \>=. 13582225/45463874 && _x_0 \<=. 44044457/20835951\l then \"versicolor\" else \"virginica\")\l = \"versicolor\" :expansions [] :rw [] :fc [])",shape=box,style=filled,fontname="courier",fontsize=14];
p_248 -> p_247 [label=""];
p_247 [label="Sat (Some let f = function\l \| _ -\> (Real.mk_of_string \"59046099/45463874\")\l \l)",shape=box,style=filled,fontname="courier",fontsize=14];
}
 Load graph Loading..

In [9]:
(* reason about weights *)
instance (fun w -> model relu w input = "versicolor")

- : t * t * t * t * t -> bool = <fun>
module CX : sig val w : t * t * t * t * t end


Instance (after 0 steps, 0.026s):
 let (w : (real * real * real * real * real)) =
 (8855.0, 11797.0,
 (Real.mk_of_string "12882364277995785799/448859531460679"), 8365.0, 0.0)

Instance call graph proof digraph "call graph" {
goal [label="let (_x_0 : real)\l = w.0 +. 31099753/100000000 *. w.1 +. -59237301/100000000 *. w.2\l +. 6692607/12500000 *. w.3 +. 17551/20000000 *. w.4\lin\llet (_x_1 : real) = if 0 \<=. _x_0 then _x_0 else 0 in\l(if _x_1 \<=. 13582225/45463874 && _x_1 \<=. 57626682/66299825 then \"setosa\"\l else\l if _x_1 \>=. 13582225/45463874 && _x_1 \<=. 44044457/20835951\l then \"versicolor\" else \"virginica\")\l= \"versicolor\"",shape=box,style=filled,color="cyan",fontname="courier",fontsize=14];
}
 Load graph Loading.. proof attempt summary full graph ground_instances 0 definitions 0 inductions 0 search_time 0.026s details Expand smt_stats num checks 2 arith-make-feasible 2 arith-max-columns 10 rlimit count 1546 mk clause 2 datatype occurs check 1 mk bool var 14 arith-lower 1 decisions 1 arith-propagations 2 propagations 1 arith-bound-propagations-cheap 2 arith-max-rows 1 datatype accessor ax 1 datatype constructor ax 1 num allocs 940022513 final checks 1 added eqs 7 arith-upper 3 memory 24.270000 max memory 24.670000 Expand start[0.026s]
 let (_x_0 : real)
 = :var_0:.0 +. :var_0:.1 *. … +. :var_0:.2 *. … +. :var_0:.3 *. …
 +. :var_0:.4 *. …
 in
 let (_x_1 : real) = if _x_0 <. 0 then 0 else _x_0 in
 let (_x_2 : real) = -20835951/25000000 *. _x_1 +. 27192873/100000000 in
 let (_x_3 : real) = -2651993/1000000 *. _x_1 +. 81521773/100000000 in
 let (_x_4 : real) = -5492791/20000000 +. -121521/100000 in
 (if _x_3 >=. _x_2 && _x_3 >=. _x_4 then "setosa"
 else if _x_2 >=. _x_3 && _x_2 >=. _x_4 then "versicolor" else "virginica")
 = "versicolor" simplify into let (_x_0 : real)
 = :var_0:.0 +. 31099753/100000000 *. :var_0:.1
 +. -59237301/100000000 *. :var_0:.2 +. 6692607/12500000 *. :var_0:.3
 +. 17551/20000000 *. :var_0:.4
in
let (_x_1 : real) = if 0 <=. _x_0 then _x_0 else 0 in
(if _x_1 <=. 13582225/45463874 && _x_1 <=. 57626682/66299825 then "setosa"
 else
 if _x_1 >=. 13582225/45463874 && _x_1 <=. 44044457/20835951
 then "versicolor" else "virginica")
= "versicolor" expansions [] rewrite_steps forward_chaining Sat (Some let (w : (real * real * real * real * real)) =
 (8855.0, 11797.0,
 (Real.mk_of_string "12882364277995785799/448859531460679"), 8365.0, 0.0)
) digraph "proof" {
p_252 [label="Start (let (_x_0 : real)\l = :var_0:.0 +. :var_0:.1 *. … +. :var_0:.2 *. …\l +. :var_0:.3 *. … +. :var_0:.4 *. …\l in\l let (_x_1 : real) = if _x_0 \<. 0 then 0 else _x_0 in\l let (_x_2 : real) = -20835951/25000000 *. _x_1 +. 27192873/100000000\l in\l let (_x_3 : real) = -2651993/1000000 *. _x_1 +. 81521773/100000000 in\l let (_x_4 : real) = -5492791/20000000 +. -121521/100000 in\l (if _x_3 \>=. _x_2 && _x_3 \>=. _x_4 then \"setosa\"\l else\l if _x_2 \>=. _x_3 && _x_2 \>=. _x_4 then \"versicolor\" else \"virginica\")\l = \"versicolor\" :time 0.026s)",shape=box,style=filled,fontname="courier",fontsize=14];
p_252 -> p_251 [label=""];
p_251 [label="Simplify (let (_x_0 : real)\l = :var_0:.0 +. 31099753/100000000 *. :var_0:.1\l +. -59237301/100000000 *. :var_0:.2\l +. 6692607/12500000 *. :var_0:.3\l +. 17551/20000000 *. :var_0:.4\l in\l let (_x_1 : real) = if 0 \<=. _x_0 then _x_0 else 0 in\l (if _x_1 \<=. 13582225/45463874 && _x_1 \<=. 57626682/66299825\l then \"setosa\"\l else\l if _x_1 \>=. 13582225/45463874 && _x_1 \<=. 44044457/20835951\l then \"versicolor\" else \"virginica\")\l = \"versicolor\" :expansions [] :rw [] :fc [])",shape=box,style=filled,fontname="courier",fontsize=14];
p_251 -> p_250 [label=""];
p_250 [label="Sat (Some let (w : (real * real * real * real * real)) =\l (8855.0, 11797.0,\l (Real.mk_of_string \"12882364277995785799/448859531460679\"), 8365.0, 0.0)\l)",shape=box,style=filled,fontname="courier",fontsize=14];
}
 Load graph Loading..

### Verify

The `verify` function takes as parameter a function representing a property and attempts to prove it; if is not proven, it gives a counter-example.

From an observation of the dataset's scatterplot, we can see that all instances with petal length < 2.5 cm belong to the setosa class. 

![scatterplot](iris_dataset_scatter_small.png)

We can ask Imandra if this property can be proven for our model:

In [48]:
verify (fun x -> is_valid_input x
        && x.petal_len <=. 2.5 /. 3.0955 -. 3.7580
        ==> model_relu x = "setosa")

- : iris_input -> bool = <fun>
module CX : sig val x : iris_input end


Counterexample (after 0 steps, 0.019s):
 let (x : iris_input) =
 {sepal_len =
 (Real.mk_of_string "304814023150662536297/405953788568996569500");
 sepal_width = 0.0; petal_len = (Real.mk_of_string "-9132889/3095500");
 petal_width = 1.0}

Refuted call graph proof digraph "call graph" {
goal [label="let (_x_0 : real) = x.sepal_len in\llet (_x_1 : real) = x.sepal_width in\llet (_x_2 : real) = x.petal_len in\llet (_x_3 : real) = x.petal_width in\llet (_x_4 : real) = 5769117/5000000 *. _x_0 in\llet (_x_5 : real) = -30127743/100000000 *. _x_1 in\llet (_x_6 : real) = 4659779/5000000 *. _x_2 in\llet (_x_7 : real) = 272461/125000 *. _x_3 in\llet (_x_8 : real)\l = if -10023211/10000000 \<=. (_x_4 +. _x_5 +. _x_6 +. _x_7)\l then 10023211/10000000 +. _x_4 +. _x_5 +. _x_6 +. _x_7 else 0\lin\lnot\l(not\l ((((((((-3 \<=. _x_0 && _x_0 \<=. 3) && -3 \<=. _x_1) && _x_1 \<=. 3)\l && -3 \<=. _x_2)\l && _x_2 \<=. 3)\l && -3 \<=. _x_3)\l && _x_3 \<=. 3)\l && _x_2 \<=. -9132889/3095500)\l \|\| (if _x_8 \<=. 13582225/45463874 && _x_8 \<=. 57626682/66299825\l then \"setosa\"\l else\l if _x_8 \>=. 13582225/45463874 && _x_8 \<=. 44044457/20835951\l then \"versicolor\" else \"virginica\")\l = \"setosa\")",shape=box,style=filled,color="cyan",fontname="courier",fontsize=14];
}
 Load graph Loading.. proof attempt summary full graph ground_instances 0 definitions 0 inductions 0 search_time 0.019s details Expand smt_stats num checks 2 arith-make-feasible 2 arith-max-columns 9 rlimit count 1904 mk clause 10 datatype occurs check 1 mk bool var 22 arith-lower 5 decisions 1 arith-propagations 2 propagations 4 arith-bound-propagations-cheap 2 arith-max-rows 1 datatype accessor ax 1 datatype constructor ax 1 num allocs 1566892367 final checks 1 added eqs 6 arith-upper 6 memory 45.090000 max memory 45.460000 Expand start[0.019s]
 let (_x_0 : real) = :var_0:.sepal_len in
 let (_x_1 : real) = :var_0:.sepal_width in
 let (_x_2 : real) = :var_0:.petal_len in
 let (_x_3 : real) = :var_0:.petal_width in
 let (_x_4 : real)
 = 10023211/10000000 +. 5769117/5000000 *. _x_0
 +. -30127743/100000000 *. _x_1 +. 4659779/5000000 *. _x_2
 +. 272461/125000 *. _x_3
 in
 let (_x_5 : real) = if _x_4 <. 0 then 0 else _x_4 in
 let (_x_6 : real) = -20835951/25000000 *. _x_5 +. 27192873/100000000 in
 let (_x_7 : real) = -2651993/1000000 *. _x_5 +. 81521773/100000000 in
 let (_x_8 : real) = -5492791/20000000 +. -121521/100000 in
 (-3 <=. _x_0
 && _x_0 <=. 3
 && -3 <=. _x_1
 && _x_1 <=. 3
 && -3 <=. _x_2 && _x_2 <=. 3 && -3 <=. _x_3 && _x_3 <=. 3)
 && _x_2 <=. ((5/2 /. 6191/2000) -. 1879/500)
 ==> (if _x_7 >=. _x_6 && _x_7 >=. _x_8 then "setosa"
 else
 if _x_6 >=. _x_7 && _x_6 >=. _x_8 then "versicolor" else "virginica")
 = "setosa" simplify into let (_x_0 : real) = :var_0:.sepal_len in
let (_x_1 : real) = :var_0:.sepal_width in
let (_x_2 : real) = :var_0:.petal_len in
let (_x_3 : real) = :var_0:.petal_width in
let (_x_4 : real) = 5769117/5000000 *. _x_0 in
let (_x_5 : real) = -30127743/100000000 *. _x_1 in
let (_x_6 : real) = 4659779/5000000 *. _x_2 in
let (_x_7 : real) = 272461/125000 *. _x_3 in
let (_x_8 : real)
 = if -10023211/10000000 <=. (_x_4 +. _x_5 +. _x_6 +. _x_7)
 then 10023211/10000000 +. _x_4 +. _x_5 +. _x_6 +. _x_7 else 0
in
not
((((((((-3 <=. _x_0 && _x_0 <=. 3) && -3 <=. _x_1) && _x_1 <=. 3)
 && -3 <=. _x_2)
 && _x_2 <=. 3)
 && -3 <=. _x_3)
 && _x_3 <=. 3)
 && _x_2 <=. -9132889/3095500)
|| (if _x_8 <=. 13582225/45463874 && _x_8 <=. 57626682/66299825 then "setosa"
 else
 if _x_8 >=. 13582225/45463874 && _x_8 <=. 44044457/20835951
 then "versicolor" else "virginica")
 = "setosa" expansions [] rewrite_steps forward_chaining Sat (Some let (x : iris_input) =
 {sepal_len =
 (Real.mk_of_string "304814023150662536297/405953788568996569500");
 sepal_width = 0.0; petal_len = (Real.mk_of_string "-9132889/3095500");
 petal_width = 1.0}
) digraph "proof" {
p_282 [label="Start (let (_x_0 : real) = :var_0:.sepal_len in\l let (_x_1 : real) = :var_0:.sepal_width in\l let (_x_2 : real) = :var_0:.petal_len in\l let (_x_3 : real) = :var_0:.petal_width in\l let (_x_4 : real)\l = 10023211/10000000 +. 5769117/5000000 *. _x_0\l +. -30127743/100000000 *. _x_1 +. 4659779/5000000 *. _x_2\l +. 272461/125000 *. _x_3\l in\l let (_x_5 : rea

### Regions visualisation

Imandra can decompose input spaces in regions. 

In [12]:
Modular_decomp.top ~assuming:"is_valid_input" "model_relu"

Error: unsupported: D.get_data: unsupported higher-order decomposition for model


However it does not work on higher-order functions. Thus we need to define a first-order hard-coded layer_0 function.

In [13]:
(* define first-order function for input layer *)

let layer_0_fo (w0, w1, w2, w3, w4) (f1, f2, f3, f4) =
    relu (w0 +. w1 *. f1 +. w2 *.f2 +. w3 *. f3 +. w4 *. f4)

let model_fo input = process_iris_input input |> layer_0_fo weights_0 |> layer_1 |> process_iris_output;;

Modular_decomp.top ~assuming:"is_valid_input" "model_fo"

val layer_0_fo : t * t * t * t * t -> t * t * t * t -> t = <fun>
val model_fo : iris_input -> string = <fun>
- : Modular_decomposition.t =
{Imandra_interactive.Modular_decomp.MD.md_session = 1i;
 md_f =
  {Imandra_surface.Uid.name = "model_fo"; id = <abstr>;
   special_tag = <abstr>; namespace = <abstr>;
   chash = Some lghTpcF1zrzF+Eu9EfRYlCTK+TqiEH+0GPYjY91EIg0;
   depth = (5i, 2i)};
 md_args = [(input : iris_input)]; md_regions = <abstr>}


Reg_id,Constraints,Invariant
7,not ((10023211/10000000 +. 5769117/5000000 *. input.sepal_len +. -30127743/100000000 *. input.sepal_width +. 4659779/5000000 *. input.petal_len +. 272461/125000 *. input.petal_width) <. 0)let (_x_0 : real) = 10023211/10000000 +. 5769117/5000000 *. input.sepal_len +. -30127743/100000000 *. input.sepal_width +. 4659779/5000000 *. input.petal_len +. 272461/125000 *. input.petal_width in (-2651993/1000000 *. _x_0 +. 81521773/100000000) >=. (-20835951/25000000 *. _x_0 +. 27192873/100000000)(-2651993/1000000 *. (10023211/10000000 +. 5769117/5000000 *. input.sepal_len +. -30127743/100000000 *. input.sepal_width +. 4659779/5000000 *. input.petal_len +. 272461/125000 *. input.petal_width) +. 81521773/100000000) >=. (-5492791/20000000 +. -121521/100000)-3/2 <=. input.sepal_leninput.sepal_len <=. 3/2-3/2 <=. input.sepal_widthinput.sepal_width <=. 3/2-3/2 <=. input.petal_leninput.petal_len <=. 3/2-3/2 <=. input.petal_widthinput.petal_width <=. 3/2,"""setosa"""
6,(10023211/10000000 +. 5769117/5000000 *. input.sepal_len +. -30127743/100000000 *. input.sepal_width +. 4659779/5000000 *. input.petal_len +. 272461/125000 *. input.petal_width) <. 0-3/2 <=. input.sepal_leninput.sepal_len <=. 3/2-3/2 <=. input.sepal_widthinput.sepal_width <=. 3/2-3/2 <=. input.petal_leninput.petal_len <=. 3/2-3/2 <=. input.petal_widthinput.petal_width <=. 3/2,"""setosa"""
5,not ((10023211/10000000 +. 5769117/5000000 *. input.sepal_len +. -30127743/100000000 *. input.sepal_width +. 4659779/5000000 *. input.petal_len +. 272461/125000 *. input.petal_width) <. 0)let (_x_0 : real) = 10023211/10000000 +. 5769117/5000000 *. input.sepal_len +. -30127743/100000000 *. input.sepal_width +. 4659779/5000000 *. input.petal_len +. 272461/125000 *. input.petal_width in (-2651993/1000000 *. _x_0 +. 81521773/100000000) >=. (-20835951/25000000 *. _x_0 +. 27192873/100000000)not ((-2651993/1000000 *. (10023211/10000000 +. 5769117/5000000 *. input.sepal_len +. -30127743/100000000 *. input.sepal_width +. 4659779/5000000 *. input.petal_len +. 272461/125000 *. input.petal_width) +. 81521773/100000000) >=. (-5492791/20000000 +. -121521/100000))let (_x_0 : real) = 10023211/10000000 +. 5769117/5000000 *. input.sepal_len +. -30127743/100000000 *. input.sepal_width +. 4659779/5000000 *. input.petal_len +. 272461/125000 *. input.petal_width in (-20835951/25000000 *. _x_0 +. 27192873/100000000) >=. (-2651993/1000000 *. _x_0 +. 81521773/100000000)(-20835951/25000000 *. (10023211/10000000 +. 5769117/5000000 *. input.sepal_len +. -30127743/100000000 *. input.sepal_width +. 4659779/5000000 *. input.petal_len +. 272461/125000 *. input.petal_width) +. 27192873/100000000) >=. (-5492791/20000000 +. -121521/100000)-3/2 <=. input.sepal_leninput.sepal_len <=. 3/2-3/2 <=. input.sepal_widthinput.sepal_width <=. 3/2-3/2 <=. input.petal_leninput.petal_len <=. 3/2-3/2 <=. input.petal_widthinput.petal_width <=. 3/2,"""versicolor"""
4,not ((10023211/10000000 +. 5769117/5000000 *. input.sepal_len +. -30127743/100000000 *. input.sepal_width +. 4659779/5000000 *. input.petal_len +. 272461/125000 *. input.petal_width) <. 0)let (_x_0 : real) = 10023211/10000000 +. 5769117/5000000 *. input.sepal_len +. -30127743/100000000 *. input.sepal_width +. 4659779/5000000 *. input.petal_len +. 272461/125000 *. input.petal_width in (-2651993/1000000 *. _x_0 +. 81521773/100000000) >=. (-20835951/25000000 *. _x_0 +. 27192873/100000000)not ((-2651993/1000000 *. (10023211/10000000 +. 5769117/5000000 *. input.sepal_len +. -30127743/100000000 *. input.sepal_width +. 4659779/5000000 *. input.petal_len +. 272461/125000 *. input.petal_width) +. 81521773/100000000) >=. (-5492791/20000000 +. -121521/100000))let (_x_0 : real) = 10023211/10000000 +. 5769117/5000000 *. input.sepal_len +. -30127743/100000000 *. input.sepal_width +. 4659779/5000000 *. input.petal_len +. 272461/125000 *. input.petal_width in (-20835951/25000000 *. _x_0 +. 27192873/100000000) >=. (-2651993/1000000 *. _x_0 +. 81521773/100000000)not ((-